1) Напишем функцию, аргументом которой будет являться sql - запрос и результатом - pd DataFrame
2) Напишем функцию, которая будет определять числовые колонки в pd DataFrame и считать для них мин, макс и среднюю
3) "Упакуем" в один проект 

In [20]:
import pandas as pd
import numpy as np
import psycopg2

In [21]:
def query_db(query, dbname = 'demo', user = 'postgres',host = '127.0.0.1',port = '5432',password = 'Lingdoc83'):
    engine = psycopg2.connect(f"dbname={dbname} user={user} host={host} port={port} password={password}")
    df = pd.read_sql(query, con=engine)
    return df

def get_numeric_stats(df, columns_to_exclude = []):
    data_types = df.dtypes
    numeric_columns = data_types.loc[(data_types == int)|(data_types == float)].index.to_list()
    for col in columns_to_exclude:
        numeric_columns.remove(col)
    num_stats = df[numeric_columns].describe().loc[['min', 'mean', 'max']]
    return num_stats

In [23]:
# Создадим запрос к БД dvdrental
query = """
----Проанализируем сколько людей перелетело из одного аэропорта в другой за месяц------
--И на какую сумму были куплены билеты--
WITH filtr AS
(
	SELECT
		--Аэропорт отправления--
		fls.departure_airport, 
		--Аэропорт прилета--
		fls.arrival_airport,
		--Вытягиваем номер месяца из даты приземления--
		TO_CHAR(fls.actual_arrival,'MM') AS per_month,
		--Подсчитываем кол-во билетов, что равняется перелетевшим людям-- 
		count(tic_f.ticket_no) AS people_flown,
		--Высчитываем сумму купленных билетов с аэропорта "А" в аэропорт "Б" в месяц--
		SUM(tic_f.amount)
	FROM
		ticket_flights AS tic_f
	LEFT JOIN 
		flights AS fls
		ON tic_f.flight_id = fls.flight_id
	--Фильтруем по статусу прибывших(что отсекает статус отмененных рейсов)--
	WHERE
		fls.status = 'Arrived'
	--Узнаем кол-во улетевших  с одного города в другой--
	-- Группируем по аэропору отправления, аэропорту прибытия и месяцу--
	GROUP BY
		fls.departure_airport, fls.arrival_airport, TO_CHAR(fls.actual_arrival,'MM')
),
--Приводим в надлежащий вид название аэропорта--
air_name AS
(
	SELECT
		airport_code,
		replace(CAST(city['ru'] AS text),'"','') || ' - ' || replace(CAST(airport_name['en'] AS text),'"','') || ' (' || airport_code || ')' AS air_town
	FROM
		airports_data
)
SELECT
	air_dep.air_town AS departure_airport,
	air_arr.air_town AS arrival_airport,
	fli.per_month,
	fli.people_flown,
	fli.sum,
	--Для отображения (Оконной функции) общего количества прибывших людей в аэропорт--
	--Хотя можно ее и не делать, условие делается ниже в CASE
	SUM(fli.people_flown) OVER (PARTITION BY air_arr.air_town) AS piople,
	--Оконная функция сумма прилетевшх людей в аэропорт c функцией CASE--
	--Пришлось оконную функцию вставить в CASE так так это уменьшает время выполнения запроса--
	--3-й временной таблицей не имеет смысла увеличивать код запроса и врямя выполнения--
	CASE WHEN (SUM(fli.people_flown) OVER (PARTITION BY air_arr.air_town))<10000 THEN 'very few'
		 WHEN (SUM(fli.people_flown) OVER (PARTITION BY air_arr.air_town))>10000 AND (SUM(fli.people_flown) OVER (PARTITION BY air_arr.air_town))<20000 THEN 'few'
		 WHEN (SUM(fli.people_flown) OVER (PARTITION BY air_arr.air_town))>20000 AND (SUM(fli.people_flown) OVER (PARTITION BY air_arr.air_town))<50000 THEN 'average'
		 ELSE 'great' 
	END AS npa
FROM 
	filtr AS fli
	LEFT JOIN
		air_name AS air_dep
		ON fli.departure_airport = air_dep.airport_code
	LEFT JOIN
		air_name AS air_arr
		ON fli.arrival_airport = air_arr.airport_code
--Оптимизировал запрос как мог, при первоначальном моем запросе время выполнения составляло 11 секунд--
"""
df = query_db(query = query)
df

OperationalError: connection to server at "127.0.0.1", port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


In [21]:
# Создадим запрос к БД demo_medium
query = 'select * from bookings.aircrafts'
dbname = 'demo'
df2 = query_db(query = query, dbname = dbname)
df2

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 61: invalid continuation byte

In [22]:
print(get_numeric_stats(df))

NameError: name 'df' is not defined